In [ ]:
from nidaqmx import Task
from nidaqmx.constants import Edge, AcquisitionType
import numpy as np

from sashimi.hardware.cameras.hamamatsu.interface import HamamatsuCamera, TriggerMode
import napari
import pyqtgraph as pg
import time

In [ ]:
max_res = 2048

camera = HamamatsuCamera(0, (max_res, max_res))

camera.exposure_time = 200
# camera.trigger_mode = TriggerMode.EXTERNAL_TRIGGER
camera.trigger_mode = TriggerMode.FREE

In [ ]:
fps = 1
ao_write_rate = 200
acq_dur = 1
n_samples = ao_write_rate*acq_dur

In [ ]:
task = Task()
task.ao_channels.add_ao_voltage_chan("Dev4/ao3", min_val=-10, max_val=10)
task.timing.cfg_samp_clk_timing(
    rate=ao_write_rate,
    source="OnboardClock",
    active_edge=Edge.RISING,
    sample_mode=AcquisitionType.CONTINUOUS,
    # sample_mode=AcquisitionType.FINITE,
    # samps_per_chan=n_samples,
)

In [ ]:
# # 5: on, 0: off

# trigger = np.array(([5, 0] * 5))
# # trigger = np.array(([5, 0] + [0] * 3) * 20)


# trigger = np.array(([5, 0] * 5))


# trigger = np.zeros(ao_write_rate//fps)
# trigger[0] = 5

# task.write(trigger)

In [ ]:
stack = []

camera.start_acquisition()
# task.write(trigger, auto_start=True)
# task.start()
start = time.time()

while time.time() - start < acq_dur:
    frame = camera.get_frames()
    stack.append(frame)

# task.stop()
camera.stop_acquisition()

In [ ]:
len(stack)

In [ ]:
stack

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
plt.imshow(frame[0])

In [ ]:
stack = np.stack([s for st in stack for s in st])
print(stack.shape)

In [ ]:
frame_arr = np.vstack([row for row in frame])

In [ ]:
frame_arr.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
plt.imshow(frame_arr)

In [ ]:
frame[0]

In [ ]:
viewer = napari.view_image(stack)

In [ ]:
start = time.time()

In [ ]:
start

In [ ]:
        # self.write_task = Task()
        # self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_ch}", min_val=-10, max_val=10)
        # self.write_task.timing.cfg_samp_clk_timing(
        #     rate=100,
        #     source="OnboardClock",
        #     active_edge=Edge.RISING,
        #     sample_mode=AcquisitionType.CONTINUOUS,
        #     samps_per_chan=1000,


In [ ]:
from nidaqmx import Task
from nidaqmx.constants import AcquisitionType, Edge
from nidaqmx.stream_readers import AnalogMultiChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter

from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QHBoxLayout, QPushButton, QMessageBox, QFileDialog
from PyQt5 import QtWidgets, QtCore
from superqt import QLabeledDoubleRangeSlider, QLabeledDoubleSlider, QLabeledSlider
import pyqtgraph as pg

import numpy as np
import sys
from skimage import io

from sashimi.hardware.cameras.hamamatsu.interface import HamamatsuCamera, TriggerMode
import flammkuchen as fl

import time

In [ ]:
class ConfocalMicroscopy(QWidget):
    def __init__(self, samp_rate=10000, amp=3, num_px=100):
        super().__init__()

        self.dev = "Dev4"
        self.ao_ch = "ao3" # cam_trigger

        self.rate = samp_rate # samples per channel per second
        self.amp = amp # scanning range in volt
        self.x_offset = 0
        self.y_offset = 0

        ######################################################
        # self.fps = 1
        # self.ao_write_rate = 100
        self.acq_dur = 1
        self.exposure_time = 10
        # self.n_samples = ao_write_rate*acq_dur
        #######################################################

        self.num_px = num_px # the number of pixels for both x and y
        self.total_px = self.num_px * self.num_px # aspect ratio = 1

        # self.write_signal = self.waveform()
        self.read_buffer = np.empty((1, self.total_px))
        self.buf_size = 5 # times bigger than one frame
        self.stack = []

        self.running = False
        self.first_img = True

        self.last_frame = []

        self.set_camera()
        self.set_gui()

    def set_camera(self):
        self.max_res = 2048
        self.camera = HamamatsuCamera(0, (self.max_res, self.max_res))
        self.camera.exposure_time = self.exposure_time
        # self.camera.trigger_mode = TriggerMode.EXTERNAL_TRIGGER
        self.camera.trigger_mode = TriggerMode.FREE

    def set_gui(self):
        self.start_button = QPushButton("Start acquisition")
        self.start_button.clicked.connect(self.toggle)

        self.save_button = QPushButton("Save last acquisition")
        # self.save_button.clicked.connect(self.save_acquisition)

        self.x_amp = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        self.x_amp.setRange(0.1, 6)
        self.x_amp.setValue(3)
        self.setFocusPolicy(QtCore.Qt.NoFocus)
        # self.x_amp.valueChanged.connect(self.update_acq_params)

        # self.srate_slider = QLabeledDoubleSlider(QtCore.Qt.Horizontal)
        # self.srate_slider.setRange(self.total_px, self.total_px*10)
        # self.srate_slider.setSingleStep(self.total_px)
        # self.srate_slider.setValue(3)
        # self.setFocusPolicy(QtCore.Qt.NoFocus)
        # self.srate_slider.valueChanged.connect(self.update_acq_params)

        self.viewer = pg.ImageView()
        self.viewer.getHistogramWidget().setHistogramRange(0, 1)
        self.viewer.ui.roiBtn.hide()
        self.viewer.ui.menuBtn.hide()

        layout = QVBoxLayout()

        layout2 = QHBoxLayout()
        layout2.addWidget(self.start_button)
        layout2.addWidget(self.save_button)
        layout.addLayout(layout2)

        layout3 = QHBoxLayout()
        layout3.addWidget(QLabel("Zoom (V)"))
        layout3.addWidget(self.x_amp)
        layout.addLayout(layout3)

        layout4 = QHBoxLayout()
        layout4.addWidget(QLabel("Sampling rate (samples/s)"))
        layout4.addWidget(QLabel("{:.3f}fps".format(self.rate/self.total_px)))
        # layout4.addWidget(self.srate_slider)
        layout.addLayout(layout4)

        layout.addWidget(self.viewer)

        self.setLayout(layout)

    def toggle(self):
        if self.running:
            self.stop()
        else:
            self.start()

    def start(self):
        self.start_button.setText("Stop acquisition")
        self.start_button.setStyleSheet("background-color: yellow")
        self.running = True

        self.set_tasks()
        # self.start_trigger()
        self.read_camera()

        # self.read_task.start()
        # self.write_task.start()

    # def start_trigger(self):
        # trigger = np.array(([5, 0] * 5))
        # self.write_task.write(trigger, auto_start=True)

    def read_camera(self):
        self.acq_start = time.time()
        self.camera.start_acquisition()
        while time.time() - self.acq_start < self.acq_dur:
            frame = self.camera.get_frames()
            print(frame)
            if not frame:
                try:
                    # self.update(np.vstack([row for row in frame]))
                    self.stack.append(frame)
                    print(len(self.stack))
                except:
                    pass

    def stop(self):
        self.start_button.setText("Start acquisition")
        self.start_button.setStyleSheet("")
        self.running = False

        self.write_task.stop()
        self.camera.stop_acquisition()
        # fl.save(r'C:\Users\portugueslab\Desktop\smth.h5',self.stack )
        # self.write_task.stop()
        # self.write_task.close()
        # self.read_task.stop()
        # self.read_task.close()

        # self.last_frame = self.reconstruct_image(self.read_buffer)
    
    def set_tasks(self):
        self.write_task = Task()
        self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/{self.ao_ch}", min_val=-10, max_val=10)
        self.write_task.timing.cfg_samp_clk_timing(
            rate=100,
            source="OnboardClock",
            active_edge=Edge.RISING,
            sample_mode=AcquisitionType.CONTINUOUS,
            # samps_per_chan=1000,
        )
        # self.write_task = Task()
        # self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/ao0", min_val=-8, max_val=8)
        # self.write_task.ao_channels.add_ao_voltage_chan(f"{self.dev}/ao3", min_val=-8, max_val=8)
        # self.write_task.timing.cfg_samp_clk_timing(
        #     rate=self.rate,
        #     source="OnboardClock",
        #     active_edge=Edge.RISING,
        #     sample_mode=AcquisitionType.CONTINUOUS,
        # )
        # self.write_task.out_stream.output_buf_size = self.total_px * 2 * self.buf_size
        # self.write_task.register_every_n_samples_transferred_from_buffer_event(self.total_px, self.write_callback)
        # self.writer = AnalogMultiChannelWriter(self.write_task.out_stream)
        # for _ in range(self.buf_size):
        #     self.writer.write_many_sample(self.write_signal)

        # self.read_task = Task()
        # self.read_task.ai_channels.add_ai_voltage_chan(f"{self.dev}/ai7", min_val=-5, max_val=5)
        # self.read_task.timing.cfg_samp_clk_timing(
        #     rate=self.rate,
        #     source="OnboardClock",
        #     active_edge=Edge.RISING,
        #     sample_mode=AcquisitionType.CONTINUOUS,
        # )
        # self.read_task.in_stream.input_buf_size = self.total_px * self.buf_size
        # self.read_task.triggers.start_trigger.cfg_dig_edge_start_trig(f"/{self.dev}/ao/StartTrigger", Edge.RISING)
        # self.read_task.register_every_n_samples_acquired_into_buffer_event(self.total_px, self.read_callback)
        # self.reader = AnalogMultiChannelReader(self.read_task.in_stream)

    # def write_callback(self, task_handle, every_n_samples_event_type, number_of_samples, callback_data):
    #     self.writer.write_many_sample(self.write_signal, timeout=1)
    #     return 0
    
    # def read_callback(self, task_handle, every_n_samples_event_type, number_of_samples, callback_data):
    #     self.reader.read_many_sample(self.read_buffer, number_of_samples, timeout=1)
    #     self.update(self.reconstruct_image(self.read_buffer))
    #     return 0
        
    def update(self, img):
        # np.vstack([row for row in frame])
        self.viewer.setImage(
            img,
            autoHistogramRange=False,
        )


if __name__ == '__main__':
    app = QApplication(sys.argv)
    main = ConfocalMicroscopy()
    main.show()
    sys.exit(app.exec_())


In [ ]:
dat = fl.load(r'C:\Users\portugueslab\Desktop\smth.h5')

In [ ]:
dat